Ноутбук для создания модели и её анализа

### Импорты

In [1]:
import pandas as pd

from src.generator.control_generator import RAGPipeline
from src.judge.control_judge import RAGJudge

pd.set_option('display.max_colwidth', None)

c:\D drive\Biocad\biocad-ml-internship\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Настройка модели для оценки

Использовался NVIDIA GeForce RTX 3050 Laptop GPU

In [2]:
model_name = "mistral-nemo"

# Инициализация RAGPipeline
rag = RAGPipeline(db_path='../db', model_name=model_name)

# Модель для сравнения ответов
judge = RAGJudge(llm_model_name=model_name)

BM25 Index built with 1947 documents.


### Тестовый набор для оценки


Загрузил JSON файл в NotebookLM и сформировал 10 вопросов и полные ответы к ним по категориям: 
1. Reasoning Evolution - переписывание вопроса таким образом, чтобы для ответа потребовалось больше рассуждений

2. Conditioning Evolution - добавление в вопрос условного элемента

3. Multi‑Context Evolution - к уже имеющемуся контексту добавляется ещё один, близкий по эмбеддингу фрагмент, и вопрос переписывается так, чтобы ответ потребовал также информацию и из нового, добавленного во фрагмент контекста

4. Conversational Evolution (in progress) - вопрос переписывается в более похожей на пользователя манере.

In [3]:
golden_set = [
    {   # 1
        "question": "Каким образом различия в экспрессии гена PLEC в разных тканях объясняют молекулярный механизм связи между болезнью Альцгеймера (БА) и сердечно-сосудистыми патологиями?",
        "ground_truth": "Интронный вариант rs11786896 гена плектина (PLEC) является общим генетическим фактором риска. Механизм заключается в тканеспецифичном влиянии: этот вариант связан с пониженной экспрессией PLEC в левом желудочке сердца (что ведет к фибрилляции предсердий и сердечной недостаточности) и одновременно с повышенной экспрессией в астроцитах головного мозга, что способствует развитию БА. Плектин критически важен для архитектуры цитоскелета, и его дисбаланс нарушает стабильность промежуточных филаментов, по-разному повреждая ткани сердца и мозга. [Источник: https://pubmed.ncbi.nlm.nih.gov/37349091, Год: 2022]\n"
    },
    {   # 2
        "question": "Если у пациента наблюдается высокая экспрессия гена растворимой эпоксидгидролазы (EPHX2), какие конкретные структурные изменения мозга он, вероятнее всего, будет иметь и почему этот ген считается безопасной мишенью для лекарств?",
        "ground_truth": "Высокие уровни EPHX2 ассоциированы с уменьшением общего объема гиппокампа, что является ключевым признаком нейродегенерации. Повышение уровня фермента sEH (кодируемого EPHX2) в гиппокампе способствует отложению Aβ и реактивации астроцитов. Эта мишень считается безопасной («druggable»), так как фенотипический анализ менделевской рандомизации не выявил побочных эффектов со стороны других систем организма при изменении уровней экспрессии этого гена. [Источник: https://pubmed.ncbi.nlm.nih.gov/37349091, Год: 2023]\n"
    },
    {   # 3
        "question": "Совместив концепцию «диабета 3-го типа» с ролью хаб-гена AKT1, объясните, как дефекты сигнализации инсулина на сетевом уровне способствуют патологическому переходу в тканях мозга при БА.",
        "ground_truth": "БА называют «диабетом 3-го типа», так как инсулинорезистентность в мозге и нарушение сигнализации IGF-1 вызывают гипометаболизм глюкозы и гибель нейронов за годы до симптомов. На сетевом уровне ген AKT1 (определенный как «переключающий» ген или switch gene) играет центральную роль в пути PI3K-AKT, который необходим для выживания нейронов и поддержания синапсов. Сбой в этом хабе блокирует нейропротекторный эффект инсулина, ускоряя гиперфосфорилирование тау-белка и синаптическую дисфункцию. [Источник: https://pubmed.ncbi.nlm.nih.gov/36499613, Год: 2022]\n [Источник: https://pubmed.ncbi.nlm.nih.gov/36207441, Год: 2022]\n"
    },
    {   # 4
        "question": "Слушай, а как наши бактерии в животе вообще могут влиять на то, что происходит в мозгу при Альцгеймере? Есть какие-то конкретные вещества, которые они выделяют, чтобы уменьшить вредный тау-белок?",
        "ground_truth": "Да, это происходит через ось «кишечник-мозг». Кишечные бактерии производят метаболиты, такие как фенилэтиламин и агматин. Экспериментально доказано, что эти вещества значительно снижают уровень гиперфосфорилированного тау-белка (p-tau181 и p-tau205) в человеческих нейронах. Агматин, например, блокирует сигнальный путь STAT3, который активируется рецептором C3AR и провоцирует патологию тау. [Источник: https://pubmed.ncbi.nlm.nih.gov/38652661, Год: 2024]\n"
    },
    {   # 5
        "question": "Почему дефицит белка SORLA (ген SORL1) приводит к парадоксальному результату: увеличению поглощения (фагоцитоза) амилоида микроглией, но при этом к неспособности мозга очиститься от него?",
        "ground_truth": "Это происходит из-за разобщения процессов захвата и деградации. Дефицит SORLA повышает количество рецепторов TREM2 на поверхности микроглии, что усиливает «заглатывание» амилоида и синаптосом. Однако внутри клетки отсутствие SORLA нарушает доставку ферментов из сети Транс-Гольджи в лизосомы через рецептор CIMPR. В итоге лизосомы увеличиваются в размерах, но становятся функционально неполноценными: они просто накапливают непереваренный «мусор», что ведет к лизосомальному стрессу и хроническому воспалению. [Источник: https://pubmed.ncbi.nlm.nih.gov/40183375, Год: 2025]\n"
    },
    {   # 6
        "question": "В рамках гипотезы «амилоидного каркаса» (amyloid scaffold), почему стратегия простого удаления амилоидных бляшек может оказаться неполной или даже проблемной с точки зрения поведения белков модуля M42?",
        "ground_truth": "Согласно этой гипотезе, бляшки сами по себе могут быть не так токсичны, как белки-сигнализаторы (например, мидкин MDK и плейотрофин PTN), которые в них накапливаются. Если терапия просто разрушает бляшку, эти накопленные белки могут резко высвободиться в микроокружение, вызывая аномальную активацию сигнальных путей в нейронах и глии. Поэтому эффективная стратегия должна быть направлена на предотвращение связывания этих модификаторов (белков M42) с амилоидом, а не только на очистку фибрилл Aβ. [Источник: https://pubmed.ncbi.nlm.nih.gov/39127040, Год: 2024]\n"
    },
    {   # 7
        "question": "Сравнивая редкие варианты, найденные методом экзомного секвенирования (WES), с белками, приоритезированными методом менделевской рандомизации (MR) с помощью AlphaFold3, какие иммунные рецепторы выходят на первый план как наиболее достоверные причинные факторы?",
        "ground_truth": "При использовании обеих высокоточных технологий ключевыми мишенями признаны TREM2 и PILRA. Анализ редких вариантов WES выделил TREM2 как критический локус риска, в то время как метод MR-SPI в сочетании с AlphaFold3 подтвердил, что миссенс-мутации в этих белках вызывают структурные изменения, напрямую влияющие на риск БА. В частности, вариант PILRA (rs1859788) считается защитным, так как снижает аффинность связывания рецептора с вредными лигандами в микроглии. [Источник: https://pubmed.ncbi.nlm.nih.gov/39129223, Год: 2024]\n[Источник: https://pubmed.ncbi.nlm.nih.gov/39637861, Год: 2024]\n[Источник: https://pubmed.ncbi.nlm.nih.gov/40050982, Год: 2025]\n"
    },
    {   # 8
        "question": "А правда, что один белок может буквально «ломать» чтение генов в мозге? И почему некоторые гены портятся быстрее других, когда этого белка становится мало?",
        "ground_truth": "Да, речь о белке TDP-43. Он контролирует правильную сборку (сплайсинг) РНК. Когда его уровень в ядре падает, в генах появляются ошибки — криптические экзоны. Гены портятся с разной скоростью из-за особенностей «посадки» белка: ген STMN2 очень чувствителен, так как TDP-43 связывается с ним в крошечном участке (22 п.н.), и даже небольшое снижение уровня белка (на 30%) вызывает сбой. У гена UNC13A сайт связывания огромный (800 п.н.), поэтому он держится дольше и ломается, только когда TDP-43 становится меньше на 50%. [Источник: https://pubmed.ncbi.nlm.nih.gov/38175301, Год: 2024]\n"
    },
    {   # 9
        "question": "Почему метаболит кинуренинового пути KYN считается более перспективным биомаркером для скрининга БА, чем его производное — хинолиновая кислота (QA), несмотря на то, что QA обладает более выраженной нейротоксичностью?",
        "ground_truth": "Несмотря на то, что хинолиновая кислота (QA) напрямую повреждает нейроны и связана с образованием бляшек и тау-танглов, она крайне плохо проникает через гематоэнцефалический барьер (ГЭБ) и существует в основном внутри мозга. Напротив, кинуренин (KYN) свободно пересекает ГЭБ через переносчики нейтральных аминокислот. Это делает KYN в плазме крови доступным и надежным индикатором метаболического дисбаланса, происходящего в центральной нервной системе, что удобнее для диагностики. [Источник: https://pubmed.ncbi.nlm.nih.gov/35096208, Год: 2022]\n"
    },
    {   # 10
        "question": "Интегрируйте данные об эндотоксинах грамотрицательных бактерий с ролью инфламмасомы NLRP3, чтобы описать молекулярный каскад, приводящий к «пироптозу» при болезни Альцгеймера.",
        "ground_truth": "Каскад начинается с повышения концентрации липополисахаридов (ЛПС) в плазме (у пациентов с БА их в 3–6 раз больше), которые действуют как «прайминг» (подготовительный сигнал) для активации NF-κB. Это запускает синтез предшественников воспалительных цитоцитов. Затем накопленные олигомеры амилоида служат «вторым сигналом», вызывая сборку инфламмасомы NLRP3 в микроглии. Это активирует фермент каспазу-1, которая не только высвобождает зрелый IL-1β, но и запускает пироптоз — форму воспалительной гибели клеток, которая массово разрушает ткани мозга и усиливает дегенерацию. [Источник: https://pubmed.ncbi.nlm.nih.gov/34876226, Год: 2021]\n[Источник: https://pubmed.ncbi.nlm.nih.gov/35250595, Год: 2022]\n[Источник: https://pubmed.ncbi.nlm.nih.gov/40427569, Год: 2025]\n"
    },
]

### Оценивание

In [4]:
df_results = judge.run_evaluation_loop(golden_set, rag)

   Evaluating case 1/10...
   Evaluating case 2/10...
   Evaluating case 3/10...
   Evaluating case 4/10...
   Evaluating case 5/10...
   Evaluating case 6/10...
   Evaluating case 7/10...
   Evaluating case 8/10...
   Evaluating case 9/10...
   Evaluating case 10/10...


In [5]:
metrics_cols = ["Cosine Similarity", "Relevance", "Depth", "Clarity", "Structure"]
print(df_results[metrics_cols].mean())

# Cosine Similarity от 0 до 1, 
# Остальные метрики от 1 до 5, где 5 — лучший результат.

Cosine Similarity    0.97109
Relevance            3.50000
Depth                3.80000
Clarity              3.40000
Structure            2.60000
dtype: float64


Результаты показывают, что есть определённая проблема с поиском узконаправленных эмбендингов (если о их тегах не спрашивается напрямую), с чем стоит поработать отдельно (условно потратить целый спринт на проработку вопроса).

От этого зависит Relevance и Depth

In [6]:
display(df_results)

,Question,Generated Answer,Ground Truth,Sources (PMID),Cosine Similarity,Relevance,Depth,Clarity,Structure
0,Каким образом различия в экспрессии гена PLEC в разных тканях объясняют молекулярный механизм связи между болезнью Альцгеймера (БА) и сердечно-сосудистыми патологиями?,"РЕЗЮМЕ: В предоставленных статьях нет точной информации о связях между различными тканями и молекулярным механизмом связи между болезнью Альцгеймера (БА) и сердечно-сосудистыми патологиями через ген PLEC.\n\nДЕТАЛЬНЫЙ АНАЛИЗ: В статье с PMID 39537608 упоминается, что ген PLEC может быть связан с риском БА и сердечно-сосудистых заболеваний через колокализованный интронный вариант rs11786896. Однако в этой статье нет информации о различиях в экспрессии гена PLEC в разных тканях.\n\nБЕЗОПАСНОСТЬ: Безопасность данных не указана в предоставленных источниках.\n\nИСТОЧНИКИ: PMID 39537608","Интронный вариант rs11786896 гена плектина (PLEC) является общим генетическим фактором риска. Механизм заключается в тканеспецифичном влиянии: этот вариант связан с пониженной экспрессией PLEC в левом желудочке сердца (что ведет к фибрилляции предсердий и сердечной недостаточности) и одновременно с повышенной экспрессией в астроцитах головного мозга, что способствует развитию БА. Плектин критически важен для архитектуры цитоскелета, и его дисбаланс нарушает стабильность промежуточных филаментов, по-разному повреждая ткани сердца и мозга. [Источник: https://pubmed.ncbi.nlm.nih.gov/37349091, Год: 2022]\n","[39537608, 39537608, 39537608, 35899035, 35899035, 35899035, 40050982, 40050982, 40050982, 36551230, 36551230, 36551230, 39890844, 39890844, 39890844, 39129223, 39129223, 39129223, 34239129, 34239129, 34239129, 38915415, 38915415, 38915415, 37349091, 37349091, 37349091, 35011725, 35011725, 35011725, 37569871, 37569871, 37569871, 38175301, 38175301, 38175301, 40304967, 40304967, 40304967, 37189731, 37189731, 37189731, 38201283, 38201283, 38201283]",0.9809,2,3,4,3
1,"Если у пациента наблюдается высокая экспрессия гена растворимой эпоксидгидролазы (EPHX2), какие конкретные структурные изменения мозга он, вероятнее всего, будет иметь и почему этот ген считается безопасной мишенью для лекарств?","**РЕЗЮМЕ**\n\nУ пациента с высокой экспрессией гена растворимой эпоксидгидролазы (EPHX2) вероятно будут наблюдаться структурные изменения в гиппокампе, такие как снижение объема гиппокампа и атрофия. Это связано с тем, что EPHX2 кодирует member of the epoxide hydrolase family, который находится в цитозоле и пероксисомах. Более высокий уровень экспрессии EPHX2 может быть связан с более низким объемом гиппокампа и ускорением атрофии гиппокампа, что может привести к болезни Альцгеймера.\n\nEPHX2 считается безопасной мишенью для лекарств, так как через phenome-wide MR не было выявлено значимых побочных эффектов, связанных с его экспрессией. Кроме того, через фармакологические исследования было показано, что генетическое удаление EPHX2 или снижение уровня sEH предотвращает накопление Aβ в мозге мышей с моделью болезни Альцгеймера и уменьшает когнитивные нарушения.\n\n**ДЕТАЛЬНЫЙ АНАЛИЗ**\n\nEPHX2 был найден через phenome-wide MR как ген, который может быть связан с направленным снижением риска заболеваний Альцгеймера в крови или ткани мозга. Более того, через фармакологические исследования было показано, что генетическое удаление EPHX2 или снижение уровня sEH предотвращает накопление Aβ в мозге мышей с моделью болезни Альцгеймера и уменьшает когнитивные нарушения. Это указывает на то, что EPHX2 может быть вовлечен в патологию болезни Альцгеймера через влияние на накопление Aβ.\n\n**БЕЗОПАСНОСТЬ**\n\nEPHX2 считается безопасной мишенью для лекарств, так как через phenome-wide MR не было выявлено значимых побочных эффектов, связанных с его экспрессией. Кроме того, через фармакологические исследования было показано, что генетическое удаление EPHX2 или снижение уровня sEH предотвращает накопление Aβ в мозге мышей с моделью болезни Альцгеймера и уменьшает когнитивные нарушения.\n\n**ИСТОЧНИКИ**\n\nP